In [211]:
pip install jupyter

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install pandas
!pip install joblib
!pip install scipy
!pip install math

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os

In [10]:
df = pd.read_pickle(r"shared/Motor_Vehicle_Collisions_-_Crashes.pkl")

In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018245 entries, 0 to 2018244
Data columns (total 28 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   CRASH DATE_CRASH TIME          datetime64[ns]
 1   BOROUGH                        object        
 2   ZIP CODE                       object        
 3   LATITUDE                       float64       
 4   LONGITUDE                      float64       
 5   LOCATION                       object        
 6   ON STREET NAME                 object        
 7   CROSS STREET NAME              object        
 8   OFF STREET NAME                object        
 9   NUMBER OF PERSONS INJURED      float64       
 10  NUMBER OF PERSONS KILLED       float64       
 11  NUMBER OF PEDESTRIANS INJURED  int64         
 12  NUMBER OF PEDESTRIANS KILLED   int64         
 13  NUMBER OF CYCLIST INJURED      int64         
 14  NUMBER OF CYCLIST KILLED       int64         
 15  NUMBER OF MOTOR

In [5]:
### Show some exploratory data analysis here
## Borough with the second highest number of crashes reported since 2012

In [218]:
#Remove NaN values in BOROUGH column and count total number of crashes by BOROUGH
BR=df.dropna(subset='BOROUGH')
print(BR['BOROUGH'].value_counts())

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64


In [219]:
#table of number of crashes and population for each borough
q3 = pd.DataFrame({'Number of Crashes':[441026,372457,313266,205345,58297],'Population':[2648452,2330295,1638281,1446788,487155]},index=['BROOKLYN','QUEENS','MANHATTAN','BRONX','STATEN ISLAND'])
print(q3)

               Number of Crashes  Population
BROOKLYN                  441026     2648452
QUEENS                    372457     2330295
MANHATTAN                 313266     1638281
BRONX                     205345     1446788
STATEN ISLAND              58297      487155


In [220]:
#number of crashes for every 100,000 people
q3['crashes for every 100,000 people']=q3['Number of Crashes']/q3['Population']*100000
print(q3)

               Number of Crashes  Population  crashes for every 100,000 people
BROOKLYN                  441026     2648452                      16652.217975
QUEENS                    372457     2330295                      15983.255339
MANHATTAN                 313266     1638281                      19121.628097
BRONX                     205345     1446788                      14193.164444
STATEN ISLAND              58297      487155                      11966.827806


In [221]:
#Remove all rows where the contributing factor vehicle 1 is not known (ie, is 'Unspecified') 
#Leaving only the crashes where only this information is available.
v1=df.dropna(subset='CONTRIBUTING FACTOR VEHICLE 1')
filtered_v1=v1[(v1['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified') & (v1['CONTRIBUTING FACTOR VEHICLE 1'] != '80') & (v1['CONTRIBUTING FACTOR VEHICLE 1'] != '1')]

In [222]:
#leading causes of crashes and the proportion of accidents
filtered_v1['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().head(5).div(1319050)

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction    0.304205
Failure to Yield Right-of-Way     0.090342
Following Too Closely             0.081473
Backing Unsafely                  0.056891
Other Vehicular                   0.047525
Name: count, dtype: float64

In [223]:
#The top 5 causes of crashes (ignoring 'Unspecified') account for what proportion of total crashes
filtered_v1['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().head(5).div(1319050).sum()

0.5804366779121338

In [224]:
#Missing values for the number of persons killed should be assumed to be equal to zero killed.
df['NUMBER OF PERSONS KILLED']=df['NUMBER OF PERSONS KILLED'].fillna(0)

In [225]:
#the total count of accidents that involved two or more fatalities
death2 = df[df['NUMBER OF PERSONS KILLED']>=2]
count_death2 = death2['NUMBER OF PERSONS KILLED'].count()
print("the total count of accidents that involved two or more fatalities:",count_death2) 

the total count of accidents that involved two or more fatalities: 88


In [226]:
#the total count of accidents that involved one or more fatalities
death1 = df[df['NUMBER OF PERSONS KILLED']>=1]
count_death1 = death1['NUMBER OF PERSONS KILLED'].count()
print("the total count of accidents that involved one or more fatalities:",count_death1) 

the total count of accidents that involved one or more fatalities: 2804


In [227]:
#calculate the total count of accidents that involved fatalities and average of fatalities in every 1000 accidents
count_death1 / df['NUMBER OF PERSONS KILLED'].count() * 1000

1.3893258747079764

In [228]:
#number of data do not have a Cross Street Name
nan_ctn = df['CROSS STREET NAME'].isna().sum()
print("number of data do not have a Cross Street Name:", nan_ctn)

number of data do not have a Cross Street Name: 755532


In [229]:
#calculate the total of accidents and proportion of accidents in the data do not have a Cross Street Name
nan_ctn/df.shape[0]

0.37435098315615795

In [230]:
#create combination of vehicle 1 and 2 and drop NaN values
df['COMBINATION VEHICLES']=df['VEHICLE TYPE CODE 1']+df['VEHICLE TYPE CODE 2']
cv=df.dropna(subset='COMBINATION VEHICLES')
cv['COMBINATION VEHICLES'].value_counts()

COMBINATION VEHICLES
SedanSedan                                                                197944
PASSENGER VEHICLEPASSENGER VEHICLE                                        193260
Station Wagon/Sport Utility VehicleStation Wagon/Sport Utility Vehicle    133780
SedanStation Wagon/Sport Utility Vehicle                                  123915
Station Wagon/Sport Utility VehicleSedan                                  123812
                                                                           ...  
PKFEDER                                                                        1
AmbulAmbul                                                                     1
Station Wagon/Sport Utility VehicleSAFET                                       1
Tractor Truck DieselAMBUL                                                      1
Station Wagon/Sport Utility Vehiclewheelchair                                  1
Name: count, Length: 6896, dtype: int64

In [231]:
#Filter crashes where the contributing factor was alcohol involvement and count how many crashes resulted in fatalities
filtered_ai = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']
filtered_ai_death=filtered_ai[filtered_ai['NUMBER OF PERSONS KILLED']>=1]

In [232]:
#total number of crashes caused by alcohol involvement and proportion of crashes that resulted in fatalities
filtered_ai_death['NUMBER OF PERSONS KILLED'].count()/filtered_ai['NUMBER OF PERSONS KILLED'].count()

0.0046638345031400075

In [233]:
#the number of crashes that occur between 4 and 7 pm and proportion
evening = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]
evening['CRASH DATE_CRASH TIME'].count()/df['CRASH DATE_CRASH TIME'].count()

0.20514010935243243

In [234]:
#Filter the dataset to include only crashes where VEHICLE TYPE CODE 1 or VEHICLE TYPE CODE 2 contains the string MOTORCYCLE
motor = df[(df['VEHICLE TYPE CODE 1'].str.contains('MOTORCYCLE'))|(df['VEHICLE TYPE CODE 2'].str.contains('MOTORCYCLE'))]

In [235]:
#check this reduced dataset for injuries greater than 0, and fatalities equal to 0 and calculate proportion
filtered_motor = motor[(motor['NUMBER OF PERSONS INJURED'] > 0) & (motor['NUMBER OF PERSONS KILLED'] == 0)]
filtered_motor['NUMBER OF PERSONS INJURED'].count()/motor['NUMBER OF PERSONS INJURED'].count()

0.5004565018912221

In [12]:
# filter crashes that involved bicycles as one of the vehicles
bic = df[(df['VEHICLE TYPE CODE 1'].str.contains('BICYCLE'))|(df['VEHICLE TYPE CODE 2'].str.contains('BICYCLE'))]
len(bic)

19108